In [5]:
print(1)


1


In [4]:
import torch as torch
import torch.nn as nn
import torch.functional as Fn
import dgl

In [3]:


from notebook.auth import passwd; passwd()

'sha1:0b41e172f42d:f51a6c7e471ca7f2f6f5ca94517c7ced172aecfc'

In [ ]:
class GATLayer(nn.Module) : 
    def __init__(self, g, in_dim, out_dim):
        super(GATLayer, self).__init__()
        self.g = g
        self.fn = nn.Linear(in_dim, out_dim, bias = False)
        self.attention_fn = nn.Linear(out_dim * 2, 1, bias = False)
        
    def edge_attention(self, edges):
        zh = torch.cat((edges.src['z'], edges.dst['z']), dim = 1)
        return {'e': nn.functional.leaky_relu(self.attention_fn(zh), 0.2), 'z' : edges.src['z']}
    
    def message_func(self, edges):
        # message UDF for equation (3) & (4)
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        # reduce UDF for equation (3) & (4)
        # equation (3)
        alpha = nn.functional.softmax(nodes.mailbox['e'], dim=1)
        # equation (4)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h': h}

    def forward(self, h) : 
        z = self.fn(h)
        self.g.ndata['z'] = z
        self.g.apply_edges(self.edge_attention)
        
        
        

In [6]:
G = dgl.DGLGraph()
G.add_nodes(10)

In [12]:
G.nodes[:].data['t'] = torch.eye(10)

In [13]:
G.nodes[:].data['t']


tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [14]:
G.add_nodes(1)



In [21]:
G.nodes[:].data['t'][10] = torch.ones(10)


In [22]:
G.nodes[:].data['t'][10]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [33]:
init = torch.ones(11)
def initializer(shape, dtype, ctx, id_range) :
    
    return init[0]

In [23]:
li = [(1,1),(2,2)]

In [24]:
for _,i in li : 
    print(i)

1
2


In [34]:
G.set_n_initializer(initializer)

In [29]:
G.nodes[:].data['test'] = torch.ones(11)

In [35]:
G.add_nodes(1)

IndexError: tuple index out of range

In [40]:
for i in range(10)[slice(3)]:
    print(i)


0
1
2


In [51]:
myslice = slice(0,5)

In [55]:
myslice.start
myslice.stop

arr  = [0,1,2,3,4,5,6,7,8,9]

In [56]:

for i in range(myslice.start, myslice.stop) : 
    print(arr[i])

0
1
2
3
4


In [57]:
arr[myslice]

[0, 1, 2, 3, 4]

In [58]:
torch.zeros((1,1), dtype=torch.float, device=torch.device('cpu'))

tensor([[0.]])

In [60]:
torch.zeros((1,1), dtype=torch.float, device=torch.device('cuda'))

tensor([[0.]], device='cuda:0')